# Image Generator

The objective of this file is o generate images from original. It will resze, rotate, flip horizontally, add gaussain noise and random brightness to generate those images and store them in folders labeled.

#### Install libraries

                pip install opencv-python numpy albumentations scikit-learn


#### Import libraries

In [ ]:
import os
import cv2
import numpy as np
import albumentations as A
from sklearn.model_selection import train_test_split
import shutil

#### File Path directory

Create a base and an output directories.
Initialize two arrays fruits (Banana, Mango, Rambutan, Strawberry, Tomato) and categories (Ripe, Unripe).


In [ ]:
# File path directory 
base_dir = "CS898 Project/Datasets" # Directory 
output_dir = "CS898 Project/TTS" # Directory

os.makedirs(output_dir, exist_ok=True)

fruits = ["Banana", "Mango", "Rambutan", "Strawberry", "Tomato"]
categories = ["Ripe", "Unripe"]


#### Transformations

Use the modal to appply the augmentation. 

In [ ]:
# Augmentation Transformations | albumentations(p = probability)
augmentations = A.Compose([
    A.GaussNoise(var_limit=(10, 50), p=0.5),  # Adds slight random noise
    A.Rotate(limit=30, p=0.7),  # Rotates between -30° and 30°
    A.HorizontalFlip(p=0.5),  # Flips image horizontally
    A.RandomBrightnessContrast(p=0.6)  # Adjusts brightness & contrast
])

In [ ]:
# Function to augment an image
def augment_image(image, count):
    augmented_images = [] # Augmented images list 
    while len(augmented_images) < count:
        augmented = augmentations(image=image)['image'] # Apply augmentation
        augmented_images.append(augmented) # append to augmented images list 
    return augmented_images

#### Categorize fruits

Loop through each fruit and category in file path 

In [ ]:
for fruit in fruits:
    for category in categories:
        input_folder = os.path.join(base_dir, fruit, category)
        output_folder = os.path.join(output_dir, fruit, category)
        os.makedirs(output_folder, exist_ok=True) # Create folder if it does not exist

        images = [f for f in os.listdir(input_folder) if f.endswith(('.jpg', '.png'))]
        
        if len(images) != 20:
            print(f"Error: {input_folder} only has {len(images)}")
            continue
        
        augmented_images = []
        
        # Load original images
        for img_name in images:
            img_path = os.path.join(input_folder, img_name)
            image = cv2.imread(img_path)
            
            # Error-handling if image cannot load
            if image is None:
                print(f"Error loading image: {img_path}")
                continue  # Skip corrupt/missing images

            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert for correct color rendering
            
            
            augmented_images.append(image) # Add originial image to list 
            
            # Ensure image dont exceed 120 
            augmented_images.extend(augment_image(image, count=5 if len(augmented_images) + 5 <= 120 else 120 - len(augmented_images)))

        # Save augmented images (ensure 120 images total)
        for i, img in enumerate(augmented_images[:120]):
            output_path = os.path.join(output_folder, f"{category}_{i+1}.jpg") # Output path directory
            cv2.imwrite(output_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))  # Convert back to BGR

        # Message report for fruit/category
        print(f"Augmented {fruit}/{category} to {len(augmented_images[:120])} images.")